# LangChain Use-Case Example: AI API Multiplexer

This example shows how you can connect to many different AI APIs using the same code by going through LangChain.

You can run this code if you have an OpenAI API key.  If you have other API keys then you can run some of the demos that use multiple AI APIs from the same code.

## Setup code

The first code cell installs the LangChain modules and retrieves AI API credentials.  Please set up Colab secrets for any of these:

* `OPENAI_API_KEY`    -- To use the OpenAI API
* `AWS_ACCESS_KEY_ID` -- To use AWS Bedrock
* `AWS_SECRET_ACCESS_KEY`
* `AWS_REGION_NAME`

In [1]:
# Load secrets
import os
from dotenv import load_dotenv
load_dotenv()

def load_environment_variables(variable_names):
    for var_name in variable_names:
        if var_name not in os.environ:
            try:
                from google.colab import userdata
                value = userdata.get(var_name)
                if value:
                    os.environ[var_name] = value
            except ImportError:
                pass
        if var_name not in os.environ:
            raise ValueError(f"{var_name} not found. Please set it in .env file or Google Colab secrets.")
        else:
            print(f"Successfully loaded {var_name} from environment variables.")

variables_to_load = [
    "OPENAI_API_KEY",
    "AWS_ACCESS_KEY_ID",
    "AWS_SECRET_ACCESS_KEY",
    "AWS_REGION_NAME"
]

load_environment_variables(variables_to_load)

Successfully loaded OPENAI_API_KEY from environment variables.
Successfully loaded AWS_ACCESS_KEY_ID from environment variables.
Successfully loaded AWS_SECRET_ACCESS_KEY from environment variables.
Successfully loaded AWS_REGION_NAME from environment variables.


In [2]:
# Install necessary libraries
!pip install langchain langchain-community langchain-core langchain-openai

# LangChain setup: Show more about what's happening as it happens.
from langchain.globals import set_debug
set_debug(True)

## Example: Minimal AI API Request

Here's a simple OpenAI API request with LangChain.

If you do this same request [with the OpenAI API](https://platform.openai.com/docs/guides/chat-completions/overview) then you get the response in `message = completion.choices[0].message.content`.  LangChain encapsulates those details so that you can think about what you're doing instead of working to accommodate the API.

In [3]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model_name="gpt-4o")

model.invoke("Is the Sagrada Família located in Barcelona?").content

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Is the Sagrada Família located in Barcelona?"
  ]
}
[llm/end] [llm:ChatOpenAI] [1.06s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Yes, the Sagrada Família is located in Barcelona, Spain. It is an iconic basilica designed by the famous architect Antoni Gaudí, and it remains one of the most visited landmarks in the city. Construction of the Sagrada Família began in 1882, and although it is still not complete, it is renowned for its unique and intricate architectural style.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "Yes, the Sagrada

'Yes, the Sagrada Família is located in Barcelona, Spain. It is an iconic basilica designed by the famous architect Antoni Gaudí, and it remains one of the most visited landmarks in the city. Construction of the Sagrada Família began in 1882, and although it is still not complete, it is renowned for its unique and intricate architectural style.'

## Example: Two Sequential Requests, Two Different Models

This example uses two different models from the same OpenAI API to handle two different parts of a larger task.

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage

chain_1 = ChatOpenAI(temperature=0.7, model_name="gpt-4o")
chain_2 = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

question = "Is the Sagrada Família located in Barcelona?"

messages = [
    HumanMessage(content=f"Question: {question}\nPlease provide a detailed explanation before answering whether the statement is true or false.")
]

detailed_response = chain_1.invoke(messages)
detailed_answer = detailed_response.content
print("Detailed answer:", detailed_answer)

messages.extend([
    AIMessage(content=detailed_answer),
    HumanMessage(content="Based on the previous messages, categorize the response strictly as either 'Yes' or 'No'.")
])

yes_no_response = chain_2.invoke(messages)
yes_no_answer = yes_no_response.content.strip()
print("Yes/No Answer:", yes_no_answer)

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Question: Is the Sagrada Família located in Barcelona?\nPlease provide a detailed explanation before answering whether the statement is true or false."
  ]
}
[llm/end] [llm:ChatOpenAI] [2.94s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Yes, the statement is true: the Sagrada Família is located in Barcelona.\n\nThe Sagrada Família, formally known as the Basílica i Temple Expiatori de la Sagrada Família, is a large unfinished Roman Catholic minor basilica in Barcelona, Catalonia, Spain. Designed by the renowned architect Antoni Gaudí, the construction of the church began in 1882, and it remains incomplete to this day, though it continues to be an active site of construction and a major tourist attraction.\n\nGaudí's work on the building is part of a UNESCO World Heritage Site, and the basilica is recognized for its unique architectural and artistic merits. Gaudí co

# Example: Use AWS Bedrock instead of OpenAI

LangChain requests to AWS Bedrock look just like LangChain requests to OpenAI.  This example of sending a request to Claude 3.5 Sonnet looks exactly like the example above that uses OpenAI.  Even though the APIs and models have different details.

Sorting out those details could consume hours of your time up front.  And complications related to managing those details could cost you even more time and delayed progress in the future.  Shifting responsibility for those details into LangChain keeps you focused on your goals.

In [5]:
from langchain_aws import ChatBedrock

model = ChatBedrock(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
)

model.invoke("Is the Sagrada Família located in Barcelona?.").content

[llm/start] [llm:ChatBedrock] Entering LLM run with input:
{
  "prompts": [
    "Human: Is the Sagrada Família located in Barcelona?."
  ]
}
[llm/end] [llm:ChatBedrock] [4.04s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Yes, the Sagrada Família is located in Barcelona, Spain.\n\nThe Sagrada Família, or to give it its full name, the Basílica i Temple Expiatori de la Sagrada Família (Basilica and Expiatory Church of the Holy Family), is a large unfinished Roman Catholic church in Barcelona. It is one of the most famous works of the Catalan architect Antoni Gaudí and has become an iconic symbol of Barcelona.\n\nConstruction of the Sagrada Família began in 1882 and is still ongoing. Despite being incomplete, it is a UNESCO World Heritage Site and one of the most visited tourist attractions in Spain. The church is located in the Eixample district of Barcelona, and its unique and intricate design makes it stand out as a landmark in the city's skyline.",


"Yes, the Sagrada Família is located in Barcelona, Spain.\n\nThe Sagrada Família, or to give it its full name, the Basílica i Temple Expiatori de la Sagrada Família (Basilica and Expiatory Church of the Holy Family), is a large unfinished Roman Catholic church in Barcelona. It is one of the most famous works of the Catalan architect Antoni Gaudí and has become an iconic symbol of Barcelona.\n\nConstruction of the Sagrada Família began in 1882 and is still ongoing. Despite being incomplete, it is a UNESCO World Heritage Site and one of the most visited tourist attractions in Spain. The church is located in the Eixample district of Barcelona, and its unique and intricate design makes it stand out as a landmark in the city's skyline."

## Example: A Sequence Of API Requests That Use Claude 3.5 Sonnet and LLama 3 8B Instruct

This code looks exactly the same as the two-step sequence above that used OpenAI.  The only thing that changed is the models.

In [10]:
from langchain_aws import ChatBedrock
from langchain_core.messages import HumanMessage, AIMessage

chain_1 = ChatBedrock(model_id="anthropic.claude-3-5-sonnet-20240620-v1:0")
chain_2 = ChatBedrock(model_id="meta.llama3-8b-instruct-v1:0")

question = "Is the Sagrada Família located in Barcelona?"

messages = [
    HumanMessage(content=f"Question: {question}\nPlease provide a detailed explanation before answering whether the statement is true or false.")
]

detailed_response = chain_1.invoke(messages)
detailed_answer = detailed_response.content
print("Detailed answer:", detailed_answer)

messages.extend([
    AIMessage(content=detailed_answer),
    HumanMessage(content="Based on the previous messages, categorize the response strictly as either 'Yes' or 'No'.")
])

yes_no_response = chain_2.invoke(messages)
yes_no_answer = yes_no_response.content.strip()
print("Yes/No Answer:", yes_no_answer)

[llm/start] [llm:ChatBedrock] Entering LLM run with input:
{
  "prompts": [
    "Human: Question: Is the Sagrada Família located in Barcelona?\nPlease provide a detailed explanation before answering whether the statement is true or false."
  ]
}
[llm/end] [llm:ChatBedrock] [6.47s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Explanation:\n\nThe Sagrada Família, officially known as the Basílica i Temple Expiatori de la Sagrada Família, is indeed located in Barcelona, Spain. It is one of the most iconic landmarks of the city and a major tourist attraction.\n\nThe construction of this extraordinary church began in 1882, and it was designed by the renowned Catalan architect Antoni Gaudí. Gaudí devoted much of his life to this project, working on it for over 40 years until his death in 1926. The basilica is known for its unique and intricate design, combining Gothic and Art Nouveau elements in a style that is distinctly Gaudí's own.\n\nSituated in the Eix

## Example: A retry loop as a safeguard layer

LLMs can be talkative.  You just want a "yes" or a "no" but they'll say things like, "The answer is: No."

We have seen how you can add a subsequent LLM call to classify a wordy answer into the "yes" or "no" that we're looking for.  But, what if the second LLM call produces something other than "yes" or "no"?

We need a retry loop as a safeguard layer.  After we call the LLM the second time, to reformat the answer, we can check to make sure we have the right format.  If not, then we can loop back and ask the second LLM a follow-up question.  Each time we do this, we send the entire message history, including the response from the more-expensive model.  You can move whole conversation histories back and forth between models like this.

In [8]:
from langchain_core.messages import HumanMessage, AIMessage
from langchain_aws import ChatBedrock

chain_1 = ChatBedrock(model_id="anthropic.claude-3-5-sonnet-20240620-v1:0")
chain_2 = ChatBedrock(model_id="mistral.mistral-small-2402-v1:0")

question = "Is the Sagrada Família located in Barcelona?"

cot_response = chain_1.invoke([HumanMessage(content=question)])
detailed_answer = cot_response.content

message_history = [
    HumanMessage(content=question),
    AIMessage(content=detailed_answer),
    HumanMessage(content="Based on the previous messages, categorize the response strictly as either 'Yes' or 'No'.")
]

max_retries = 3
retry_count = 0

while retry_count < max_retries:
    yes_no_response = chain_2.invoke(message_history)
    yes_no_answer = yes_no_response.content.strip().lower()
    
    if yes_no_answer in ["yes", "no"]:
        print(f"Yes/No Answer: {yes_no_answer.capitalize()}")
        break
    else:
        print(f"Attempt {retry_count + 1}: Invalid response '{yes_no_answer}'. Retrying...")
        retry_count += 1
        
        if retry_count < max_retries:
            message_history.append(HumanMessage(content="Your previous response was not valid. Please answer ONLY with 'Yes' or 'No', without any additional text or punctuation."))

if retry_count == max_retries:
    print("Failed to get a valid Yes/No answer after maximum retries.")

[llm/start] [llm:ChatBedrock] Entering LLM run with input:
{
  "prompts": [
    "Human: Is the Sagrada Família located in Barcelona?"
  ]
}
[llm/end] [llm:ChatBedrock] [4.87s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Yes, the Sagrada Família is located in Barcelona, Spain. It is one of the most famous and iconic landmarks in the city and a major tourist attraction. The full name of the church is the Basílica i Temple Expiatori de la Sagrada Família (Basilica and Expiatory Church of the Holy Family).\n\nSome key points about the Sagrada Família:\n\n1. It was designed by the renowned Catalan architect Antoni Gaudí.\n\n2. Construction began in 1882 and is still ongoing.\n\n3. It is considered Gaudí's masterpiece and combines Gothic and Art Nouveau elements.\n\n4. The church is expected to be completed around 2026, the centenary of Gaudí's death.\n\n5. It was consecrated as a minor basilica by Pope Benedict XVI in 2010.\n\n6. The Sagrada Família is a